# Job crawler - Gupy  

#### Load Libs

In [1]:
import requests
import pandas as pd
from datetime import date
import time
import random
from google.cloud import bigquery
import pandas_gbq
import warnings
warnings. filterwarnings("ignore")

#### Load raw data

In [2]:
def normalize_columns(df):
    # Renomeando colunas
    df_new = df[['portal', 'term','date_extraction', 'name', 'description', \
            'careerPageName', 'type', 'publishedDate', \
            'applicationDeadline', 'isRemoteWork', 'city', \
                'state', 'country', 'jobUrl', 'careerPageUrl']]
    
    df_new.rename(columns={'careerPageName': 'company_name', 
                            'publishedDate': 'published_date', 
                            'applicationDeadline': 'application_deadline', 
                            'isRemoteWork': 'is_remote_work', 
                            'jobUrl': 'job_url', 
                            'careerPageUrl': 'company_url'}, inplace=True)
    return df_new

In [3]:
# defining the url to get the data
url = "https://portal.api.gupy.io/api/job"

# defining the job title to search for
search_terms = ['data', 'dados', 'product', 'engenharia', 'back-end', 'front-end']

# defining the dataframe to store the data
df_jobs_raw = pd.DataFrame()

# defining today to use in some cases
today = date.today()

for term in search_terms:
    querystring = {"name":term,"offset":"1","limit":"5000"}

    time.sleep(random.randint(3, 6))
    r = requests.request("GET", url, params=querystring)

    data = r.json()

    df_term = pd.json_normalize(data, record_path =['data'])
    df_term['portal'] = "Gupy"
    df_term['term'] = term
    df_term['date_extraction'] = today

    df_jobs_raw = pd.concat([df_jobs_raw, df_term], ignore_index=True)
    df_jobs_normalized = normalize_columns(df_jobs_raw)

df_jobs = df_jobs_normalized.copy()

display(df_jobs.head())

,portal,term,date_extraction,name,description,company_name,type,published_date,application_deadline,is_remote_work,city,state,country,job_url,company_url
0,Gupy,data,2023-04-20,Analista Data Center JR - Virtualização (VMWare),SOBRE A EMPRESA:A Unidade de Negócio de Cloud ...,Telefónica Tech,vacancy_type_effective,2023-04-19T12:05:04.822Z,None,False,Santana de Parnaíba,São Paulo,Brasil,https://ttech.gupy.io/job/eyJqb2JJZCI6NDcwMzc0...,https://ttech.gupy.io/eyJzb3VyY2UiOiJndXB5X3Bv...
1,Gupy,data,2023-04-20,ANALISTA II BIG DATA - GOIÂNIA/GO II || POI - ...,"Para compor ainda mais esse time de sucesso, b...",Evolução profissional a gente faz em casa,vacancy_type_effective,2023-04-18T19:14:00.918Z,None,False,Goiânia,Goiás,Brasil,https://poi.gupy.io/job/eyJqb2JJZCI6NDY5ODUyNy...,https://poi.gupy.io/eyJzb3VyY2UiOiJndXB5X3Bvcn...
2,Gupy,data,2023-04-20,Estágio em Data Quality,Quer fazer parte de um time alta performance?&...,Neogrid Carreiras,vacancy_type_internship,2023-04-18T17:48:42.627Z,None,True,Porto Alegre,Rio Grande do Sul,Brasil,https://neogridcarreiras.gupy.io/job/eyJqb2JJZ...,https://neogridcarreiras.gupy.io/eyJzb3VyY2UiO...
3,Gupy,data,2023-04-20,Assistente de Data Quality [LETT],Quer fazer parte de um time alta performance?&...,Neogrid Carreiras,vacancy_type_effective,2023-04-18T17:27:18.497Z,None,True,São Paulo,São Paulo,Brasil,https://neogridcarreiras.gupy.io/job/eyJqb2JJZ...,https://neogridcarreiras.gupy.io/eyJzb3VyY2UiO...
4,Gupy,data,2023-04-20,Assistente de Data Quality (Manutenção),Quer fazer parte de um time alta performance?&...,Neogrid Carreiras,vacancy_type_effective,2023-04-18T17:25:43.453Z,None,False,São Paulo,São Paulo,Brasil,https://neogridcarreiras.gupy.io/job/eyJqb2JJZ...,https://neogridcarreiras.gupy.io/eyJzb3VyY2UiO...


#### Understanding and cleaning data

In [4]:
df_jobs.describe()

,portal,term,date_extraction,name,description,company_name,type,published_date,application_deadline,is_remote_work,city,state,country,job_url,company_url
count,1895,1895,1895,1895,1895,1895,1895,1895,87,1895,1895,1895,1895,1895,1895
unique,1,6,1,1588,1766,772,12,1853,26,2,250,26,9,1853,786
top,Gupy,engenharia,2023-04-20,Product Owner,"Falar da MRV&amp;CO (MRV, Urba, Luggo, Sensia ...",Itaú Unibanco,vacancy_type_effective,2023-03-17T17:57:26.262Z,2023-04-30T00:00:00.000Z,False,São Paulo,São Paulo,Brasil,https://agasus.gupy.io/job/eyJqb2JJZCI6NDQ2OTk...,https://vemproitau.gupy.io/eyJzb3VyY2UiOiJndXB...
freq,1895,727,1895,18,11,67,1165,2,14,1300,501,885,1713,2,67


##### Change datatypes

In [5]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895 entries, 0 to 1894
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   portal                1895 non-null   object
 1   term                  1895 non-null   object
 2   date_extraction       1895 non-null   object
 3   name                  1895 non-null   object
 4   description           1895 non-null   object
 5   company_name          1895 non-null   object
 6   type                  1895 non-null   object
 7   published_date        1895 non-null   object
 8   application_deadline  87 non-null     object
 9   is_remote_work        1895 non-null   bool  
 10  city                  1895 non-null   object
 11  state                 1895 non-null   object
 12  country               1895 non-null   object
 13  job_url               1895 non-null   object
 14  company_url           1895 non-null   object
dtypes: bool(1), object(14)
memory usage: 2

In [6]:
# change columns to datetime
cols = ['date_extraction', 'application_deadline', 'published_date']

for col in cols:
    df_jobs[col] = pd.to_datetime(df_jobs[col], format="%Y-%m-%d")

In [7]:
# change columns to category
cols = ['term', 'type', 'city', 'state', 'country']

for col in cols:
    df_jobs[col] = df_jobs[col].astype('category')

In [8]:
df_jobs['published_date']

0      2023-04-19 12:05:04.822000+00:00
1      2023-04-18 19:14:00.918000+00:00
2      2023-04-18 17:48:42.627000+00:00
3      2023-04-18 17:27:18.497000+00:00
4      2023-04-18 17:25:43.453000+00:00
                     ...               
1890          2022-02-21 19:08:33+00:00
1891          2022-01-14 20:28:01+00:00
1892          2021-08-24 17:31:51+00:00
1893          2021-06-04 22:33:05+00:00
1894          2018-05-24 17:22:36+00:00
Name: published_date, Length: 1895, dtype: datetime64[ns, UTC]

In [9]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895 entries, 0 to 1894
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   portal                1895 non-null   object             
 1   term                  1895 non-null   category           
 2   date_extraction       1895 non-null   datetime64[ns]     
 3   name                  1895 non-null   object             
 4   description           1895 non-null   object             
 5   company_name          1895 non-null   object             
 6   type                  1895 non-null   category           
 7   published_date        1895 non-null   datetime64[ns, UTC]
 8   application_deadline  87 non-null     datetime64[ns, UTC]
 9   is_remote_work        1895 non-null   bool               
 10  city                  1895 non-null   category           
 11  state                 1895 non-null   category           
 12  countr

##### Check missing values

In [10]:
df_jobs.isna().sum()

portal                     0
term                       0
date_extraction            0
name                       0
description                0
company_name               0
type                       0
published_date             0
application_deadline    1808
is_remote_work             0
city                       0
state                      0
country                    0
job_url                    0
company_url                0
dtype: int64

##### Fixing duplicated values

In [11]:
# check for duplicated rows
df_jobs.duplicated().sum()

0

In [12]:
# see if any data is duplicated for job_url column and drop it
print(f"Duplicated before: {df_jobs.duplicated(subset='job_url').sum()}" )
df_jobs.drop_duplicates(subset='job_url', inplace=True)

print(f"Duplicated after: {df_jobs.duplicated(subset='job_url').sum()}")

Duplicated before: 42
Duplicated after: 0


In [13]:
df_jobs.sample(20)

,portal,term,date_extraction,name,description,company_name,type,published_date,application_deadline,is_remote_work,city,state,country,job_url,company_url
686,Gupy,dados,2023-04-20,Você tem interesse em ingressar na área de Eng...,Nossa nave tem um lugar reservado pra você! Va...,Semantix,vacancy_type_talent_pool,2020-12-05 21:45:14+00:00,NaT,True,São Paulo,São Paulo,Brasil,https://semantix.gupy.io/job/eyJqb2JJZCI6NjA0N...,https://semantix.gupy.io/eyJzb3VyY2UiOiJndXB5X...
937,Gupy,engenharia,2023-04-20,Estagiário em Engenharia de Produto,Vaga para o time de Pesquisa &amp; Desenvolvim...,BYD Brasil,vacancy_type_internship,2023-04-19 19:50:32.353000+00:00,NaT,False,Campinas,São Paulo,Brasil,https://bydbrasil.gupy.io/job/eyJqb2JJZCI6NDcw...,https://bydbrasil.gupy.io/eyJzb3VyY2UiOiJndXB5...
957,Gupy,engenharia,2023-04-20,Tutor Facilitador Estratégico - Engenharia,Queremos recebê-lo (a) no Universo do Conhecim...,Unicesumar,vacancy_type_effective,2023-04-19 13:00:05.520000+00:00,NaT,False,Apucarana,Paraná,Brasil,https://unicesumaroficial.gupy.io/job/eyJqb2JJ...,https://unicesumaroficial.gupy.io/eyJzb3VyY2Ui...
1222,Gupy,engenharia,2023-04-20,PROFESSOR(A) DE MUSCULAÇÃO - CLUBE DE ENGENHARIA,A&nbsp;Bluefit&nbsp;é uma rede de academias br...,Bluefit Academia,vacancy_type_effective,2023-03-29 13:23:29.439000+00:00,NaT,False,Brasilia,Distrito Federal,Brasil,https://bluefit.gupy.io/job/eyJqb2JJZCI6NDUxOD...,https://bluefit.gupy.io/eyJzb3VyY2UiOiJndXB5X3...
1739,Gupy,back-end,2023-04-20,Desenvolvedor Back End SR (foco em .NET),A Gipsyy é uma startup de tecnologia com o DNA...,Gipsyy,vacancy_type_effective,2023-01-19 14:54:08.705000+00:00,NaT,False,São Paulo,São Paulo,Brasil,https://vempragipsyy.gupy.io/job/eyJqb2JJZCI6M...,https://vempragipsyy.gupy.io/eyJzb3VyY2UiOiJnd...
1327,Gupy,engenharia,2023-04-20,BANCO DE TALENTOS: ESTAGIÁRIO DE ENGENHARIA ME...,Atuamos no desenvolvimento de projetos para os...,SANDECH Engenharia,vacancy_type_internship,2023-03-13 19:17:08.064000+00:00,NaT,False,Rio de Janeiro,Rio de Janeiro,Brasil,https://sandech.gupy.io/job/eyJqb2JJZCI6NDQ0Mz...,https://sandech.gupy.io/eyJzb3VyY2UiOiJndXB5X3...
638,Gupy,dados,2023-04-20,Administrador de Banco de Dados (DBA),"Olá, nós somos a Jazz!&nbsp;&nbsp;Somos uma pl...",Jazz Tech,vacancy_type_effective,2022-10-20 17:00:07.927000+00:00,NaT,False,São Paulo,São Paulo,Brasil,https://jazztech.gupy.io/job/eyJqb2JJZCI6MzI0N...,https://jazztech.gupy.io/eyJzb3VyY2UiOiJndXB5X...
1081,Gupy,engenharia,2023-04-20,ANALISTA DE ENGENHARIA DE PROCESSOS SR,Atuar na engenharia de processos para novas fe...,Itaú Unibanco,vacancy_type_temporary,2023-04-11 15:46:17.075000+00:00,NaT,False,Sao Paulo,São Paulo,Brasil,https://vemproitau.gupy.io/job/eyJqb2JJZCI6NDU...,https://vemproitau.gupy.io/eyJzb3VyY2UiOiJndXB...
550,Gupy,dados,2023-04-20,Analista de Dados,"A Leads2b é uma empresa de tecnologia, SaaS B2...",Leads2b,vacancy_type_effective,2023-01-26 16:37:02.521000+00:00,NaT,False,Curitiba,Paraná,Brasil,https://leads2b.gupy.io/job/eyJqb2JJZCI6Mzk5MT...,https://leads2b.gupy.io/eyJzb3VyY2UiOiJndXB5X3...
979,Gupy,engenharia,2023-04-20,Estágio Engenharia Civil/Arquitetura - Rio de ...,"Falar da MRV&amp;CO (MRV, Urba, Luggo, Sensia ...",Venha construir o futuro com a gente!,vacancy_type_internship,2023-04-18 13:17:03.760000+00:00,NaT,False,Rio de Janeiro,Rio de Janeiro,Brasil,https://vagas-mrveco.gupy.io/job/eyJqb2JJZCI6N...,https://vagas-mrveco.gupy.io/eyJzb3VyY2UiOiJnd...


## Load it
Send the data to Bigquery

In [16]:
# use local json file to authenticate
SERVICE_ACCOUNT_JSON = r"..\job-tracker-384222-84c152151770.json"
client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_JSON)

pandas_gbq.to_gbq(df_jobs, 'job_portal_data.gupy', project_id='job-tracker-384222', if_exists='append')

100%|██████████| 1/1 [00:00<?, ?it/s]
